# Strava scraper

This is the first attempt at scraping all cycling routes posted on strava located in **Spain**. We only need to download the *gpx* files.

In [2]:
#Importing libraries.

import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [5]:
#Initializing our webdriver (Chrome).

driver = webdriver.Chrome()

In [7]:
#Entering a ride.

driver.get('https://www.strava.com/activities/5798394213')

In [8]:
#Clicking the three-dot menu.

menu = driver.find_element_by_xpath('/html/body/div[2]/div[3]/nav/div/div')
menu.click()

In [9]:
#Downloading the gpx.

gpx = driver.find_element_by_xpath('/html/body/div[2]/div[3]/nav/div/div/ul/li[3]')
gpx.click()

In [27]:
#Obtaining the ride type. This will only match "Bicicleta" if it's a bike ride.

ridetype = driver.find_element_by_xpath('//*[@id="heading"]/header/h2/span').text[13:]
ridetype

'Bicicleta'

In [28]:
#Obtaining the location:

location = driver.find_element_by_xpath('//*[@id="heading"]/div/div/div[1]/div/div/span').text
location

'Londres, Reino Unido'

In [ ]:
#Naming schema for download links.

'http://strava.com/activities/5798394100/export_gpx'

Names of spanish CCAAs as they are used on **Strava**:

Asturias
Cataluña
Madrid
Aragón